## Máster en Big Data y Data Science

### Metodologías de gestión y diseño de proyectos de big data

#### AP2 - Descubrimiento de grupos en los datos

---

En esta libreta se generan los modelos de clusterización sobre el dataset final del escenario para su posterior evaluación y análisis. Se utilizará como herramienta de soporte a mlflow para el registro completo de la experimentación. 

---

In [17]:
# Importación de librerías

import pandas as pd
# Se importa mlflow para registro de la experimentación 
import mlflow
# Se importan los métodos a utilizar para clusterizar
from sklearn.cluster import DBSCAN, KMeans, AgglomerativeClustering, MeanShift, SpectralClustering, AffinityPropagation, BisectingKMeans
# Se importan las métricas a utilizar para evaluar el proceso
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.mixture import GaussianMixture


----

##### Lectura del dataset

In [6]:
df = pd.read_csv("../../../data/final/datos_clusterizacion.csv", sep=";")
df.drop(['id_cliente'], axis=1, inplace=True)
df.head(5)

,edad,importe_solicitado,duracion_credito,antiguedad_empleado,ingresos,pct_ingreso,tasa_interes,estado_credito,antiguedad_cliente,gastos_ult_12m,...,estado_cliente_ACTIVO,estado_cliente_PASIVO,genero_F,genero_M,nivel_educativo_DESCONOCIDO,nivel_educativo_POSGRADO_COMPLETO,nivel_educativo_POSGRADO_INCOMPLETO,nivel_educativo_SECUNDARIO_COMPLETO,nivel_educativo_UNIVERSITARIO_COMPLETO,nivel_educativo_UNIVERSITARIO_INCOMPLETO
0,22,35000,3,123.0,59000,0.59,16.02,1,36.0,1088.0,...,True,False,False,True,False,False,False,False,True,False
1,21,1000,2,5.0,9600,0.10,11.14,0,39.0,1144.0,...,True,False,False,True,False,False,False,True,False,False
2,23,35000,2,4.0,65500,0.53,15.23,1,36.0,1887.0,...,True,False,False,True,False,False,False,False,True,False
3,24,35000,4,8.0,54400,0.55,14.27,1,54.0,1314.0,...,True,False,False,True,True,False,False,False,False,False
4,21,2500,2,2.0,9900,0.25,7.14,1,34.0,1171.0,...,True,False,True,False,False,False,False,True,False,False


### Prueba # 1

----

##### Configuración de la experimentación

In [7]:
# Se genera el experimento en mlflow
exp_name = 'Experimentación clustering Extra #1'
exp_id = mlflow.create_experiment(name=exp_name)

In [8]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id, run_name="Kmeans - K=4"):
    # Selección y configuración de la técnica
    modelo_clusters = KMeans(n_clusters=4)
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)

    # Se registra el parámetro K
    mlflow.log_param('Valor K', 4)

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    mlflow.sklearn.log_model(trained_model, "Kmeans_K4")

    # Se finaliza el registro
    mlflow.end_run()

Para contabilizar los elementos por cluster se utilizan las labels generadas

In [9]:
# Se preserva el conjunto de datos original para evitar problemas
df_kmeans_k4 = df.copy()
# Se agregan las labels generadas
df_kmeans_k4['cluster'] = cluster_labels
df_kmeans_k4['cluster'].value_counts()

cluster
0    4190
2    3847
1     686
3     162
Name: count, dtype: int64

In [ ]:
df_kmeans_k2.to_csv("salidak2.csv", sep=";". index=False)

Se pasa a probar otro método (clustering jerárquico)

In [10]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id, run_name="AGClustering - K=4"):
    # Selección y configuración de la técnica
    modelo_clusters = AgglomerativeClustering(n_clusters=4)
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)

    # Se registra el parámetro K
    mlflow.log_param('Valor K', 4)

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    mlflow.sklearn.log_model(trained_model, "AGC_K4")

    # Se finaliza el registro
    mlflow.end_run()

2024/07/21 18:17:43 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


Para contabilizar los elementos por cluster se utilizan las labels generadas

In [11]:
# Se preserva el conjunto de datos original para evitar problemas
df_agc_k4 = df.copy()
# Se agregan las labels generadas
df_agc_k4['cluster'] = cluster_labels
df_agc_k4['cluster'].value_counts()

cluster
0    4697
2    3817
1     307
3      64
Name: count, dtype: int64

MeanShift

In [12]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id, run_name="MeanShift"):
    # Selección y configuración de la técnica
    modelo_clusters = MeanShift()
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)

    # No hay parámetros por registrar

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    mlflow.sklearn.log_model(trained_model, "MS")

    # Se finaliza el registro
    mlflow.end_run()

Para contabilizar los elementos por cluster se utilizan las labels generadas

In [13]:
# Se preserva el conjunto de datos original para evitar problemas
df_ms = df.copy()
# Se agregan las labels generadas
df_ms['cluster'] = cluster_labels
df_ms['cluster'].value_counts()

cluster
0    8519
1     306
2      39
3      11
4       6
5       3
6       1
Name: count, dtype: int64

SpectralClustering

In [14]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id, run_name="SpectralClustering K4"):
    # Selección y configuración de la técnica
    modelo_clusters = SpectralClustering(n_clusters = 4)
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)

    # No hay parámetros por registrar

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    mlflow.sklearn.log_model(trained_model, "SC-k4")

    # Se finaliza el registro
    mlflow.end_run()

d:\DOCUMENTOS\VIU\modulos\13MBID\actividad1\13MBID-AP1\.venv\Lib\site-packages\sklearn\manifold\_spectral_embedding.py:301: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
d:\DOCUMENTOS\VIU\modulos\13MBID\actividad1\13MBID-AP1\.venv\Lib\site-packages\sklearn\base.py:1474: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (4). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
2024/07/21 18:30:11 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


In [15]:
# Se preserva el conjunto de datos original para evitar problemas
df_sc4 = df.copy()
# Se agregan las labels generadas
df_sc4['cluster'] = cluster_labels
df_sc4['cluster'].value_counts()

cluster
0    8883
1       2
Name: count, dtype: int64

BisectingKMeans

In [18]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id, run_name="BisectingKMeans K=4"):
    # Selección y configuración de la técnica
    modelo_clusters = BisectingKMeans(n_clusters = 4)
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)

    # No hay parámetros por registrar

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    mlflow.sklearn.log_model(trained_model, "BK-4")

    # Se finaliza el registro
    mlflow.end_run()

In [19]:
# Se preserva el conjunto de datos original para evitar problemas
df_bk4 = df.copy()
# Se agregan las labels generadas
df_bk4['cluster'] = cluster_labels
df_bk4['cluster'].value_counts()

cluster
1    4229
0    4053
3     454
2     149
Name: count, dtype: int64

### Prueba # 2

In [20]:
# Se genera el experimento en mlflow
exp_name = 'Experimentación clustering Extra #2'
exp_id2 = mlflow.create_experiment(name=exp_name)



- Kmeans


In [21]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id2, run_name="Kmeans - K=6"):
    # Selección y configuración de la técnica
    modelo_clusters = KMeans(n_clusters=6)
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)

    # Se registra el parámetro K
    mlflow.log_param('Valor K', 6)

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    mlflow.sklearn.log_model(trained_model, "Kmeans_K6")

    # Se finaliza el registro
    mlflow.end_run()

In [22]:
# Se preserva el conjunto de datos original para evitar problemas
df_kmeans_k6 = df.copy()
# Se agregan las labels generadas
df_kmeans_k6['cluster'] = cluster_labels
df_kmeans_k6['cluster'].value_counts()

cluster
1    3557
4    3217
3     918
2     841
5     251
0     101
Name: count, dtype: int64

- Clustering Jerárquico

In [23]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id2, run_name="AGClustering - K=6"):
    # Selección y configuración de la técnica
    modelo_clusters = AgglomerativeClustering(n_clusters=6)
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)

    # Se registra el parámetro K
    mlflow.log_param('Valor K', 6)

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    mlflow.sklearn.log_model(trained_model, "AGC_K6")

    # Se finaliza el registro
    mlflow.end_run()

2024/07/21 20:03:42 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


In [24]:
# Se preserva el conjunto de datos original para evitar problemas
df_agc_k6 = df.copy()
# Se agregan las labels generadas
df_agc_k6['cluster'] = cluster_labels
df_agc_k6['cluster'].value_counts()

cluster
4    3576
2    3105
0    1121
5     712
1     307
3      64
Name: count, dtype: int64

In [25]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id2, run_name="BisectingKMeans K=6"):
    # Selección y configuración de la técnica
    modelo_clusters = BisectingKMeans(n_clusters = 6)
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)
  
    # Se registra el parámetro K
    mlflow.log_param('Valor K', 6)

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    mlflow.sklearn.log_model(trained_model, "BK-6")

    # Se finaliza el registro
    mlflow.end_run()

In [26]:
# Se preserva el conjunto de datos original para evitar problemas
df_bk6 = df.copy()
# Se agregan las labels generadas
df_bk6['cluster'] = cluster_labels
df_bk6['cluster'].value_counts()

cluster
3    3298
5    2368
4    1861
2     755
1     454
0     149
Name: count, dtype: int64

### Prueba # 3

In [27]:
# Se genera el experimento en mlflow
exp_name = 'Experimentación clustering Extra #3'
exp_id3 = mlflow.create_experiment(name=exp_name)

In [28]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id3, run_name="Kmeans - K=8"):
    # Selección y configuración de la técnica
    modelo_clusters = KMeans(n_clusters=8)
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)

    # Se registra el parámetro K
    mlflow.log_param('Valor K', 8)

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    mlflow.sklearn.log_model(trained_model, "Kmeans_K8")

    # Se finaliza el registro
    mlflow.end_run()

In [29]:
# Se preserva el conjunto de datos original para evitar problemas
df_k8 = df.copy()
# Se agregan las labels generadas
df_k8['cluster'] = cluster_labels
df_k8['cluster'].value_counts()

cluster
7    2711
6    2118
0    2009
2     664
5     598
3     571
1     154
4      60
Name: count, dtype: int64

In [32]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id3, run_name="Kmeans - K=3"):
    # Selección y configuración de la técnica
    modelo_clusters = KMeans(n_clusters=3)
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)

    # Se registra el parámetro K
    mlflow.log_param('Valor K', 3)

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    mlflow.sklearn.log_model(trained_model, "Kmeans_K3")

    # Se finaliza el registro
    mlflow.end_run()

In [33]:
# Se preserva el conjunto de datos original para evitar problemas
df_k3 = df.copy()
# Se agregan las labels generadas
df_k3['cluster'] = cluster_labels
df_k3['cluster'].value_counts()

cluster
2    4701
0    3878
1     306
Name: count, dtype: int64

Con las metricas comparadas en mlfow Kmeans con 3 clusters tiene mejor evaluación

In [36]:
df_k3.to_csv("../../../data/final/resultados_kmeans_k3-p3.csv", sep=";", index=False)

In [30]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id3, run_name="BisectingKMeans K=8"):
    # Selección y configuración de la técnica
    modelo_clusters = BisectingKMeans(n_clusters = 8)
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)
  
    # Se registra el parámetro K
    mlflow.log_param('Valor K', 8)

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    mlflow.sklearn.log_model(trained_model, "BK-8")

    # Se finaliza el registro
    mlflow.end_run()

In [31]:
# Se preserva el conjunto de datos original para evitar problemas
df_bk8 = df.copy()
# Se agregan las labels generadas
df_bk8['cluster'] = cluster_labels
df_bk8['cluster'].value_counts()

cluster
3    2675
2    2538
1    1129
4     759
5     619
0     562
6     454
7     149
Name: count, dtype: int64

In [37]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id3, run_name="BisectingKMeans K=3"):
    # Selección y configuración de la técnica
    modelo_clusters = BisectingKMeans(n_clusters = 3)
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)
  
    # Se registra el parámetro K
    mlflow.log_param('Valor K', 3)

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    mlflow.sklearn.log_model(trained_model, "BK-3")

    # Se finaliza el registro
    mlflow.end_run()

In [39]:
# Se preserva el conjunto de datos original para evitar problemas
df_bk3 = df.copy()
# Se agregan las labels generadas
df_bk3['cluster'] = cluster_labels
df_bk3['cluster'].value_counts()

cluster
2    4229
1    4053
0     603
Name: count, dtype: int64